In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.cluster import KMeans
from sklearn.gaussian_process.kernels import Matern, RBF

import plotly.express as px

import plotly.graph_objects as go

from scipy.linalg import toeplitz, block_diag

from spe.mse_estimator import ErrorComparer
from spe.data_generation import (
    gen_rbf_X,
    gen_matern_X,
    create_clus_split,
    gen_cov_mat,
)
from spe.relaxed_lasso import RelaxedLasso
from spe.estimators import (
    kfoldcv,
    kmeanscv,
    better_test_est_split,
    cp_corr_adaptive_smoother_train_test,
    bag_kfoldcv,
    bag_kmeanscv,
)

import os

In [2]:
niter = 50

n=30**2
p=200
s=30

k=5

alpha = .05
delta = .5

lambd = .2
nboot = 100

snr = 0.4

noise_kernel = 'matern'
noise_length_scale = 1.
noise_nu = 0.5

X_kernel = 'rbf'
X_length_scale = 5.
X_nu = 2.5

idx = -1

savedir='~'

In [3]:
# noise_kernel = kernel
# noise_length_scale = length_scale
# noise_nu = nu

if X_kernel is None:
    X_kernel = kernel
if X_length_scale is None:
    X_length_scale = noise_length_scale
if X_nu is None:
    X_nu = noise_nu


In [4]:
models = RelaxedLasso(lambd=lambd)

ests = [better_test_est_split,
        cp_corr_adaptive_smoother_train_test,
#         cp_corr_adaptive_smoother_train_test, 
        kfoldcv,
        kmeanscv]
est_kwargs = [{},
              {'alpha': alpha, 
               'use_trace_corr': True, 
               'full_refit':False,
               'nboot': nboot}, 
#               {'alpha': alpha, 
#                'use_trace_corr': False, 
#                'full_refit':False,
#                'nboot': nboot},
              {'k': k},
              {'k': k}]

In [5]:
if not os.path.exists(os.path.expanduser(savedir)):
    os.makedirs(os.path.expanduser(savedir))

params = pd.DataFrame({'niter': niter,
                       'n': n, 
                       'p': p, 
                       's': s,
                       'k': k,
                       'snr': snr,
                       'lambd': lambd,
                       'nboot': nboot,
                       'alpha': alpha,
                       'delta': delta,
                       'nk': noise_kernel, 
                       'nls': noise_length_scale, 
                       'nn': noise_nu, 
                       'xk': X_kernel,
                       'xls': X_length_scale,
                       'xn': X_nu}, index=[idx])
params.to_csv(os.path.expanduser(savedir + 'params.csv'))
dffp = os.path.expanduser(savedir + "err_df.csv")
# barfp = os.path.expanduser(savedir + 'barchart.jpeg')

In [6]:
err_cmp = ErrorComparer()

In [7]:
nx = ny = int(np.sqrt(n))
xs = np.linspace(0, 10, nx)
ys = np.linspace(0, 10, ny)
c_x, c_y = np.meshgrid(xs, ys)
c_x = c_x.flatten()
c_y = c_y.flatten()
coord = np.stack([c_x, c_y]).T

In [8]:
if noise_kernel == 'rbf':
    Sigma_t = gen_cov_mat(c_x, c_y, RBF(length_scale=noise_length_scale))
elif noise_kernel == 'matern':
    Sigma_t = gen_cov_mat(c_x, c_y, Matern(length_scale=noise_length_scale, nu=noise_nu))
else:
    Sigma_t = np.eye(n)
    
Cov_st = delta*Sigma_t
Sigma_s = delta*Sigma_t \
            + (1-delta)*gen_cov_mat(c_x, 
                                    c_y, 
                                    Matern(
                                        length_scale=noise_length_scale,
                                        nu=noise_nu
                                    )
                                   )
Sigma_t = delta*Sigma_t \
            + (1-delta)*gen_cov_mat(c_x, 
                                    c_y, 
                                    RBF(length_scale=noise_length_scale)
                                   )
# Cov_st = np.zeros((n,n))

if noise_kernel == 'rbf' or noise_kernel == 'matern':
    Chol_t = np.linalg.cholesky(Sigma_t)
    Chol_s = np.linalg.cholesky(Sigma_s)
else:
    Chol_t = np.eye(n)
    Chol_s = np.eye(n)

In [9]:
assert(np.allclose(Sigma_t, Chol_t @ Chol_t.T))

In [10]:
# assert(np.allclose(Cov_st, Chol_t @ Chol_t.T - (1-delta)*np.eye(n)))

In [11]:
if X_kernel == 'rbf':
    X = gen_rbf_X(c_x, c_y, p)
elif X_kernel == 'matern':
    X = gen_matern_X(c_x, c_y, p, length_scale=X_length_scale, nu=X_nu)
else:
    X = np.random.randn(n,p)

beta = np.zeros(p)
idx = np.random.choice(p,size=s)
beta[idx] = np.random.uniform(-1,1,size=s)

In [12]:
(test_err,
#  rnd_err,
 trc_err,
 kfcv_err,
 spcv_err) = err_cmp.compare(models,
                             ests,
                             est_kwargs,
                             niter=niter,
                             n=n,
                             p=p,
                             s=p,
                             snr=snr, 
                             X=X,
                             beta=beta,
                             coord=coord,
                             Chol_t=Chol_t,
                             Chol_s=Chol_s,
                             Cov_st=Cov_st,
                             delta=delta,
                             tr_idx=None,
                             fair=False,
                             )

0
0.5833333333333334
10
20
30
40


In [13]:
risk = test_err.mean()
risk_kfcv = kfcv_err.mean()
risk_spcv = spcv_err.mean()
# risk_rnd = rnd_err.mean()
risk_trc = trc_err.mean()
# risk, risk_kfcv, risk_spcv, risk_lin

In [14]:
save_df = pd.DataFrame({
#     'Rand_WR': (rnd_err.T),
    'Trace_WR': (trc_err.T),
    'KFCV': (kfcv_err.T),
    'SPCV': (spcv_err.T),
    'Test': (test_err.T)
})
save_df.to_csv(dffp)

In [15]:
df = pd.DataFrame({
#     'Rand_WR': (rnd_err.T),
    'Trace_WR': (trc_err.T),
    'KFCV': (kfcv_err.T),
    'SPCV': (spcv_err.T)
})

In [16]:
df.mean(), (df/risk).mean()

(Trace_WR    0.070067
 KFCV        0.061764
 SPCV        0.134894
 dtype: float64,
 Trace_WR    0.997993
 KFCV        0.879736
 SPCV        1.921359
 dtype: float64)

In [17]:
df.var(), (df/risk).var()

(Trace_WR    0.000315
 KFCV        0.000191
 SPCV        0.001074
 dtype: float64,
 Trace_WR    0.063845
 KFCV        0.038753
 SPCV        0.217843
 dtype: float64)

In [19]:
fig = go.Figure()
fig.add_trace(go.Bar(
    name='Control',
#     x=['Rand_WR', 'Trace_WR', 'KFCV', 'SPCV'],
    x=['Trace_WR', 'KFCV', 'SPCV'],
    y=(df/risk).mean(),
    marker_color=px.colors.qualitative.Plotly,
    text=np.around((df/risk).mean(),3),
    textposition='outside',
    error_y=dict(
        type='data',
        color='black',
        symmetric=False,
        array=(df/risk).quantile(.75) - (df/risk).mean(),
        arrayminus=(df/risk).mean() - (df/risk).quantile(.25))
#         array=(df/risk).mean() + (df/risk).std(),
#         arrayminus=(df/risk).mean() - (df/risk).std())
))
# fig.add_trace(go.Bar(
#     name='Experimental',
#     x=['GenCp', 'KFCV', 'SPCV'], y=(df/risk).mean(),
#     error_y=dict(type='data', array=[1, 2])
# ))
# fig.update_layout(barmode='group')
fig.add_hline(y=1., line_color='red')
fig.update_layout(
#     title=f"RL_\u03B1{alpha}_\u03B4{delta}_snr{snr}_nk{noise_kernel}_nls{noise_length_scale}_nv{noise_nu}_Xk{X_kernel}_Xls{X_length_scale}_Xv{X_nu}",
    title=f"Relaxed Lasso: Trace vs 5 Fold CV, Dependent Y, Y*",
    xaxis_title="Method",
    yaxis_title="Relative MSE",
#     legend_title="Legend Title",
#     font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="RebeccaPurple"
#     )
)
barfp = os.path.expanduser(savedir + fig.layout.title['text'] + ".jpeg")
# fig.write_image(os.path.expanduser(barfp))
fig.show()